# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [4]:
# importing the necessary libraries
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas

import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [5]:
# Reading the Mapbox API key
load_dotenv()
MAPBOX_API_KEY = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(MAPBOX_API_KEY)

# Import Data

In [6]:
# Importing the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
neighborhoods_coordinates = pd.read_csv(file_path, index_col="Neighborhood")

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [7]:
# Defining the Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units_per_year = sfo_data.groupby("year").mean()   
    housing_units_per_year = housing_units_per_year.drop(["gross_rent","sale_price_sqr_foot"], axis=1)    
    Dash_housing_units_per_year= housing_units_per_year.plot.line
    return Dash_housing_units_per_year 

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    average_gross_rent = sfo_data.groupby("year").mean()
    average_gross_rent = average_gross_rent.drop(["housing_units","sale_price_sqr_foot"], axis=1)    
    Dash_average_gross_rent = average_gross_rent.plot.line    
    return Dash_average_gross_rent

def average_sales_price():
    """Average Sales Price Per Year."""
    average_sales_price = sfo_data.groupby("year").mean()
    average_sales_price = average_sales_price.drop(["gross_rent","housing_units"], axis=1)    
    Dash_sales_price_sqr_foot = average_sales_price.plot.line
    return Dash_sales_price_sqr_foot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    average_price_by_neighborhood=sfo_data.groupby(["year","neighborhood"]).mean()
    average_price_by_neighborhood = pd.DataFrame(average_price_by_neighborhood)
    Dash_average_prices_neighborhood =average_price_by_neighborhood.plot.line
    return Dash_average_prices_neighborhood

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    df_temp = sfo_data.groupby("neighborhood").mean()
    top_most_expensive_neighborhoods = df_temp.nlargest(10,'sale_price_sqr_foot')
    Dash_top_most_expensive_neighborhoods = top_most_expensive_neighborhoods.plot.line
    return Dash_top_most_expensive_neighborhoods

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""
    df_all_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_all_neighborhoods.reset_index(inplace=True)
    df_all_neighborhoods.rename(columns={"neighborhood": "Neighborhood"}, inplace=True)
    Dash_df_all_neighborhoods = df_all_neighborhoods.plot.line
    return Dash_df_all_neighborhoods

def parallel_coordinates(neighborhoods_coordinates):
    """Parallel Coordinates Plot."""
    Dash_parallel_coordinates = px.parallel_coordinates(neighborhoods_coordinates)
    return Dash_parallel_coordinates

def parallel_categories(neighborhoods_coordinates):
    """Parallel Categories Plot."""
    Dash_parallel_categories = px.parallel_categories(neighborhoods_coordinates)
    return Dash_parallel_categories

def neighborhood_map(neighborhoods_coordinates):
    """Neighborhood Map."""
    neighorbood_map = px.scatter_mapbox(
    neighborhoods_coordinates,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=15,
    zoom=11,
    hover_name="Neighborhood",
    title="Average Sale Price Per Square Foot and Gross Rent in San Francisco",
    )
    Dash_neighborhood_map = neighborhood_map
    return Dash_neighborhood_map

def sunburst(neighborhoods_coordinates):
    """Sunburst Plot."""
    df=neighborhoods_coordinates
    fig=px.sunburst(df,path=["Neighboorhood","gross_rent"],
                        values="sale_price_sqr_foot")    
    

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [8]:
# Create a Title for the Dashboard
#dashboard

HOUSING_UNITS_PER_YEAR = pn.Column(housing_units_per_year())
AVERAGE_GROSS_RENT = pn.Column(average_gross_rent())
AVERAGE_SALES_PRICE = pn.Column(average_sales_price())
AVERAGE_PRICE_BY_NEIGHBORHOOD = pn.Column(average_price_by_neighborhood())
TOP_MOST_EXPENSIVE_NEIGHBORHOODS = pn.Column(top_most_expensive_neighborhoods())
MOST_EXPENSIVE_NEIGHBORHOODS_RENT_SALES = pn.Column(most_expensive_neighborhoods_rent_sales())

tabs = pn.Tabs((housing_units_per_year),
                (average_gross_rent()),
                (average_sales_price()),
                (average_price_by_neighborhood()),
                (top_most_expensive_neighborhoods()),
                (most_expensive_neighborhoods_rent_sales())
)


## Serve the Panel Dashboard

In [112]:
# Serve the# dashboard
#dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [9]:
housing_units_per_year()

<bound method PlotAccessor.line of <pandas.plotting._core.PlotAccessor object at 0x0000011B10356488>>

In [56]:
average_gross_rent()

<bound method PlotAccessor.line of <pandas.plotting._core.PlotAccessor object at 0x000002978E0A43C8>>

In [57]:
 average_sales_price()

<bound method PlotAccessor.line of <pandas.plotting._core.PlotAccessor object at 0x000002978E0B5748>>

In [58]:
average_price_by_neighborhood()

<bound method PlotAccessor.line of <pandas.plotting._core.PlotAccessor object at 0x000002978E050508>>

In [59]:
top_most_expensive_neighborhoods()

<bound method PlotAccessor.line of <pandas.plotting._core.PlotAccessor object at 0x000002978E066688>>

In [60]:
most_expensive_neighborhoods_rent_sales()

<bound method PlotAccessor.line of <pandas.plotting._core.PlotAccessor object at 0x000002978C43A7C8>>

In [15]:
neighborhood_map()

TypeError: neighborhood_map() missing 1 required positional argument: 'neighborhoods_coordinates'

In [25]:
parallel_categories()

TypeError: parallel_categories() missing 1 required positional argument: 'neighborhoods_coordinates'

In [26]:
parallel_coordinates()

TypeError: parallel_coordinates() missing 1 required positional argument: 'neighborhoods_coordinates'

In [27]:
sunburst()

TypeError: sunburst() missing 1 required positional argument: 'neighborhoods_coordinates'